In [ ]:
import pandas as pd

folder_path = 'DSS_DATASET_UPDATED.xlsx'
df = pd.read_excel(folder_path, sheet_name = 'O3')
# O2 = Purpose of Trips 
# O3 =  Number by Destination - Region
# O5 = Mode of Departure

df

In [ ]:
df.rename(columns={'Unnamed: 1': 'Year'}, inplace=True)
df.rename(columns={'Unnamed: 2': 'Month'}, inplace=True)

df

In [ ]:
df = df[3:].reset_index(drop=True)

df

In [ ]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
index = 0
for month in df['Month']:
    if index >= 2:
        if month not in months:
            df = df.drop(index)
    index += 1

df = df.reset_index(drop=True)
df

In [ ]:
years = []
count = 0
for Num in df['Year']:
    if not pd.isnull(Num):
        years.append(int(Num))
    else:
        if count > 0:
            years.append(int(years[count-1]))
        else:
            years.append(0)
    count += 1

df['Year'] = years

df

In [ ]:
df['Year'] = df['Year'].astype(str)
df['Month'] = df['Month'].astype(str)

df

In [ ]:
df['Year - Month'] = df['Year'] + ' - ' + df['Month']

df

In [ ]:
col = df['Year - Month']
df.drop(labels=['Year - Month'], axis=1, inplace=True)
df.insert(0, 'Year - Month', col)

df

In [ ]:
df.drop(['(Last update: 26-06-2023)', 'Year', 'Month'], axis=1, inplace=True)

df

In [ ]:
df = df.transpose()

df

In [ ]:
df = df.rename(columns={0: 'Region'})
df = df.rename(columns={1: 'Citizenship'})

df

In [ ]:
df['Region'] = df['Region'].str.split('\n').str[0]
df['Citizenship'] = df['Citizenship'].str.split('\n').str[0]

df

In [ ]:
df = df.reset_index(drop=True)

df

In [ ]:
citizenships = ['Saudi', 'Non-Saudi']
index = 0
for val in df['Citizenship']:
    if index >= 1:
        if val not in citizenships:
            df = df.drop(index)
    index += 1
        
df

In [ ]:
regions = []
count = 0
for val in df['Region']:
    if not pd.isnull(val):
        regions.append(val)
    else:
        if count > 0:
            regions.append(regions[count-1])
        else:
            regions.append(None)
    count += 1

df['Region'] = regions

df = df.reset_index(drop=True)
df

In [ ]:
df = df.iloc[0:15].reset_index(drop=True)

df

In [ ]:
df['Region, Citizenship'] = df['Region'] + ', ' + df['Citizenship']

df

In [ ]:
col = df['Region, Citizenship']
df.drop(labels=['Region, Citizenship'], axis=1, inplace=True)
df.insert(0, 'Region, Citizenship', col)

df

In [ ]:
df.drop(['Region', 'Citizenship'], axis=1, inplace=True)

df

In [ ]:
df = df.transpose()
df = df.reset_index(drop=True)
df

In [ ]:
df.columns = df.iloc[0]
df = df[1:].reset_index(drop=True)

df

In [ ]:
fact_table = df.rename(columns={'0 - Month, 0 - nan': 'Year - Month'})

fact_table

In [ ]:
fact_table = pd.melt(fact_table, id_vars=['Year - Month'], var_name="Region, Citizenship", value_name="Number_of_Tourists")

fact_table

In [ ]:
fact_table['Year'] = fact_table['Year - Month'].str.split(' - ').str[0]
fact_table['Month'] = fact_table['Year - Month'].str.split(' - ').str[1]

fact_table.drop(columns=['Year - Month'], inplace=True)

fact_table

In [ ]:
quarters = []
for month in fact_table['Month']:
    if month in months[0:3]:
        quarters.append('Q1')
    elif month in months[3:6]:
        quarters.append('Q2')
    elif month in months[6:9]:
        quarters.append('Q3')
    elif month in months[9:]:
        quarters.append('Q4')
    else:
        quarters.append(None)

fact_table['Quarter'] = quarters

fact_table

In [ ]:
fact_table['Region'] = fact_table['Region, Citizenship'].str.split(', ').str[0]
fact_table['Citizenship'] = fact_table['Region, Citizenship'].str.split(', ').str[1]

fact_table.drop(columns=['Region, Citizenship'], inplace=True)

fact_table

In [ ]:
fact_table['Region'] = fact_table['Region'].str.lstrip()
fact_table['Region'] = fact_table['Region'].str.rstrip()

fact_table

In [ ]:
def format_region(region):
    words = region.split()
    formatted_words = []
    for word in words:
        if len(word) <= 3:
            formatted_word = word
        else:
            formatted_word = word.capitalize()
        formatted_words.append(formatted_word)
    return ' '.join(formatted_words)

fact_table['Region'] = fact_table['Region'].apply(format_region)

fact_table

In [ ]:
date_dim = pd.read_csv('Tourism_Date_Dim.csv')

date_dim

In [ ]:
fact_table['Year'] = fact_table['Year'].astype(int) 

fact_table

In [ ]:
fact_table = pd.merge(fact_table, date_dim, on=['Year', 'Quarter', 'Month'], how='left')

fact_table

In [ ]:
citizenship_dim = pd.read_csv('Tourism_Citizenship_Dim.csv')

citizenship_dim

In [ ]:
fact_table = pd.merge(fact_table, citizenship_dim, on='Citizenship', how='left')

fact_table

In [ ]:
loc_dim = pd.read_csv('Tourism_Location_Dim.csv')

loc_dim

In [ ]:
fact_table = pd.merge(fact_table, loc_dim, on='Region', how='left')

fact_table

In [ ]:
fact_table = fact_table[['Date_ID', 'Citizenship_ID', 'Location_ID', 'Number_of_Tourists']]

fact_table

In [ ]:
fact_table = fact_table.sort_values(by=['Date_ID', 'Citizenship_ID', 'Location_ID'])

fact_table

In [ ]:
fact_table['Number_of_Tourists'] = fact_table['Number_of_Tourists'].apply(lambda x: '{:,.3f}'.format(x) if x != 0 else x)

fact_table

In [ ]:
fact_table['Number_of_Tourists'] = fact_table['Number_of_Tourists'].apply(lambda x: x.replace('.', '') if x != 0 else x)

fact_table

In [ ]:
fact_table['Number_of_Tourists'] = fact_table['Number_of_Tourists'].astype(str)

fact_table

In [ ]:
fact_table['Number_of_Tourists'] = fact_table['Number_of_Tourists'].apply(lambda x: x.lstrip('0') if x != '0' else x)

fact_table

In [ ]:
fact_table['Number_of_Tourists'] = fact_table['Number_of_Tourists'].apply(lambda x: '{:,.0f}'.format(int(x)) if x != '0' else x)

fact_table

In [ ]:
fact_table.to_csv('Tourism_Location_Fact.csv', index=False)